In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Dropout,LeakyReLU,Flatten,AveragePooling2D

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, confusion_matrix
import itertools

In [ ]:
#Declare variables and images folders
image_size = 256
batch_size = 32
input_shape = (image_size, image_size, 3)
train_folders = '../data/train/'
validation_folders = '../data/validation/'
test_folders = '../data/test/'

quantity_tr = {} 
quantity_te = {}

for folder in os.listdir(train_folders):
    quantity_tr[folder] = len(os.listdir(train_folders+folder))

for folder in os.listdir(validation_folders):
    quantity_te[folder] = len(os.listdir(validation_folders+folder))
    
quantity_train = pd.DataFrame(list(quantity_tr.items()), index=range(0,len(quantity_tr)), columns=['class','count'])
quantity_test = pd.DataFrame(list(quantity_te.items()), index=range(0,len(quantity_te)), columns=['class','count'])

figure, ax = plt.subplots(1,2,figsize=(20,5))
sns.barplot(x='class',y='count',data=quantity_train,ax=ax[0])
sns.barplot(x='class',y='count',data=quantity_test,ax=ax[1])

print("Number of images in the train set : ", sum(quantity_tr.values()))
print("Number of images in the validation set ; ",sum(quantity_te.values()))
number_of_images_in_test_set = len(os.listdir(test_folders))
print("Number of images in test set : ",number_of_images_in_test_set)

plt.show()

In [ ]:
#load datasets with dataaugmentation initialized for training set
train_datagen = ImageDataGenerator(rescale=1./255.,
                                   rotation_range=25,
                                   fill_mode='nearest',
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   brightness_range=(0.5, 1.5),
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True)

train_dataset = train_datagen.flow_from_directory(train_folders,
                                                  target_size=(image_size, image_size),
                                                  batch_size=batch_size,
                                                  shuffle=True,
                                                  class_mode='categorical')

validation_datagen = ImageDataGenerator(rescale = 1.0/255.)
validation_dataset = validation_datagen.flow_from_directory(validation_folders,
                                                            target_size=(image_size, image_size),
                                                            batch_size=batch_size,
                                                            shuffle=False,
                                                            class_mode='categorical')

test_datagen = ImageDataGenerator(rescale = 1.0/255)
test_dataset = test_datagen.flow_from_directory(test_folders,
                                                target_size=(image_size, image_size),
                                                batch_size=batch_size,
                                                shuffle=False,
                                                class_mode='categorical')


In [ ]:
#view the classes in the dataset
train_dataset.class_indices

In [ ]:
inv_map_classes = {v: k for k, v in validation_dataset.class_indices.items()}
print(validation_dataset.class_indices)
print(inv_map_classes)

In [ ]:
#function to show images or classify them if predict_using_model is true

def show_few_images(number_of_examples=2, predict_using_model=None):
    figure1, ax1 = plt.subplots(number_of_examples,len(os.listdir(train_folders)), figsize=(20,4*number_of_examples))
    ax1 = ax1.reshape(-1)
    axoff_fun = np.vectorize(lambda ax:ax.axis('off'))
    axoff_fun(ax1)
    axs = 0
    for i, folder in enumerate(os.listdir(train_folders)):
        image_ids = os.listdir(os.path.join(train_folders,folder))
        for j in [random.randrange(0, len(image_ids)) for i in range(0,number_of_examples)]:
            img_path = os.path.join(train_folders,folder,image_ids[j])
            display = plt.imread(img_path)
            img = cv2.resize(cv2.imread(img_path), (image_size, image_size))
            img_normalized = img/255
            plt.axis('off')
            ax1[axs].imshow(display)
            title = 'True:'+folder
            if(predict_using_model):
                predicted_classname = inv_map_classes[np.argmax(best_model.predict(np.array([img_normalized])))]
                title = title+'\nPredict :'+predicted_classname
            ax1[axs].set_title(title)
            axs=axs+1

In [ ]:
show_few_images()

In [ ]:
#save history to file to be able to load it later
def save_history(history, model_name):
    #convert the history.history dict to a pandas DataFrame:     
    hist_df = pd.DataFrame(history.history) 

    # save to json:  
    hist_json_file = model_name+'_history.json' 
    with open(hist_json_file, mode='w') as f:
        hist_df.to_json(f)

    # or save to csv: 
    hist_csv_file = model_name+'_history.csv'
    with open(hist_csv_file, mode='w') as f:
        hist_df.to_csv(f)
        
#plot accuracy from model created, can be modified to plot from file history       
def plot_accuracy_from_history(history):
    color = sns.color_palette()
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    
    epochs = range(len(acc))

    sns.lineplot(epochs, acc, label='Training Accuracy')
    sns.lineplot(epochs, val_acc,label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()
    plt.figure()
    plt.show()
    
#plot the loss
def plot_loss_from_history(history):
    color = sns.color_palette()
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    epochs = range(len(loss))
    
    sns.lineplot(epochs, loss,label='Training Loss')
    sns.lineplot(epochs, val_loss, label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.figure()
    plt.show()
    
#things to do after training    
def do_history_stuff(history, history_file_name):
    save_history(history, history_file_name)
    plot_accuracy_from_history(history)
    plot_loss_from_history(history)

In [ ]:
#get class weight
class_weights = class_weight.compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(train_dataset.classes),
                                        y = train_dataset.classes                                                    
                                    )
train_class_weights = dict(zip(np.unique(train_dataset.classes), class_weights))
print(train_class_weights)

In [ ]:
tf.keras.backend.clear_session()
inception_epoch = 200

In [ ]:
#get model and view it summary
InceptionV3_model = InceptionV3(input_shape=input_shape, weights='imagenet', include_top=False)

InceptionV3_model.summary()

In [ ]:
#freeze upper layers of model
for layer in InceptionV3_model.layers[:249]:
   layer.trainable = False
for layer in InceptionV3_model.layers[249:]:
   layer.trainable = True

In [ ]:
#load ad add layers to the model
InceptionV3_last_output = InceptionV3_model.output
InceptionV3_maxpooled_output = Flatten()(InceptionV3_last_output)
InceptionV3_x = Dense(1024, activation='relu')(InceptionV3_maxpooled_output)
InceptionV3_x = Dropout(0.5)(InceptionV3_x)
InceptionV3_x = Dense(3, activation='softmax')(InceptionV3_x)

InceptionV3_x_final_model = Model(inputs=InceptionV3_model.input,outputs=InceptionV3_x)

InceptionV3_x_final_model.summary()

In [ ]:
base_learning_rate = 0.0001
number_of_epochs = inception_epoch
#compile model with optimizer

# Adam(learning_rate=base_learning_rate, decay=base_learning_rate/40)
InceptionV3_x_final_model.compile(optimizer=SGD(learning_rate=base_learning_rate, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

InceptionV3_filepath = 'inceptionv3_'+'-saved-model-{epoch:02d}-loss-{loss:.2f}.h5'

#save model as checkpoint
checkpoint = ModelCheckpoint(InceptionV3_filepath, monitor='val_accuracy', mode='max',verbose=1, save_best_only=True)
# restore_best_weights=True verbose=1,mode='max'

#stop model if loss does not reduce in 25 epochs
early_stopping = EarlyStopping(monitor='loss', patience=25)
learning_rate_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=7, verbose=1, min_delta=1e-4, mode='min')

callback_list = [checkpoint, early_stopping, learning_rate_reduce]

InceptionV3_history = InceptionV3_x_final_model.fit(train_dataset, epochs = number_of_epochs,
                                                    validation_data = validation_dataset,
                                                    class_weight = train_class_weights,
                                                    callbacks=callback_list, verbose=1)

best_model = InceptionV3_x_final_model
model_history = InceptionV3_history

do_history_stuff(InceptionV3_history, 'InceptionV3_model') #plot accuracy and loss after training


In [ ]:
#evaluate accuracy of model
test_score = best_model.evaluate(test_dataset)

print("")
print("[INFO] Accuracy: {:.2f}%".format(test_score[1] * 100))
print("[INFO] Loss: ",test_score[0])

In [ ]:
#plot accuracy and loss from training history
plot_accuracy_from_history(model_history)
plot_accuracy_from_history(model_history)

In [ ]:
#Plot the confusion matrix.
def plot_confusion_matrix(cm, classes, normalize=True, title='Confusion matrix', cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize=(7,7))
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm = np.around(cm, decimals=2)
        cm[np.isnan(cm)] = 0.0
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    thresh = cm.max() / 2.
    
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j], horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")
    
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    

target_names = []
for key in train_dataset.class_indices:
    target_names.append(key)
    
#Confusion Matrix

Y_pred = best_model.predict(test_dataset)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
cm = confusion_matrix(test_dataset.classes, y_pred)
plot_confusion_matrix(cm, target_names)

#Print Classification Report
print('Classification Report')
print(classification_report(test_dataset.classes, y_pred, target_names=target_names))

In [ ]:
#show a few images and their prediction from validation set
show_few_images(3, True)

In [ ]:
image_size = 256
model = load_model("inceptionv3_-saved-model-76-loss-0.37.h5")
img_path = 'test.jpg'
test_image = image.load_img(img_path, target_size =(image_size,image_size))
plt.imshow(test_image)
plt.axis('off')
plt.show()
test_image = image.img_to_array(test_image)
# Expanding the 3-d image to 4-d image.
# The dimensions will be Batch, Height, Width, Channel
test_image = np.expand_dims(test_image, axis = 0)
# Predicting the final class
predict = model.predict(test_image)
predictions = predict[0].argmax()
confidence = round(100 * (np.max(predict[0])), 2)
if predictions == 0:
    print("The test image has: Age Related Macular Degeneration with confidence of:", confidence, "%")
elif predictions == 1:
    print("The test image has: Diabetic Retinopathy confidence of:", confidence, "%")
else:
    print("The test image is: healthy: with confidence of:",confidence ,"%")

In [ ]:
model_version=max([int(i) for i in os.listdir("../models/retrain") + [0]])+1
best_model.save(f"../models/retrain/{model_version}")